# Notebook Goals
The goal of this notebook is to replicate the result of HOLC Chicago. Perhaps do the expansion job on the other python notebooks.

In [88]:
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

%matplotlib inline

import seaborn as sns
sns.set_style("whitegrid")
sns.set_context('paper')

paper_cmap = {"A":"#5FCE72","B":"#0BC0ED","C":"#FFD419","D":"#FF4B19"}
paper_colors = LinearSegmentedColormap('paper_colors', paper_cmap)

import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
import glob
import requests

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
from shapely import wkt

import psycopg2  # (if it is postgres/postgis)
conn = psycopg2.connect(database="chicago_fha", user="wonyoungso", password="",
    host="localhost")
cursor = conn.cursor()


from sqlalchemy import create_engine
engine = create_engine('postgresql://wonyoungso:@localhost:5432/chicago_fha')

from geoalchemy2 import Geometry, WKTElement


import warnings
warnings.filterwarnings('ignore')


from patsylearn import PatsyModel, PatsyTransformer

import libpysal as ps

import statsmodels.api as sm
import statsmodels.stats as st


In [89]:
df_final = gpd.read_file('data/chicago_census_historical/chicago_census_historical.shp')

df_final = df_final.rename(columns={'asian': 'asian',
 'asian_pe_1': 'asian_perc_norm',
 'asian_perc': 'asian_perc',
 'black': 'black',
 'black_norm': 'black_norm',
 'black_pe_1': 'black_perc_norm',
 'black_perc': 'black_perc',
 'college': 'college',
 'college__1': 'college_perc_norm',
 'college_pe': 'college_perc',
 'gisjoin': 'gisjoin',
 'gisjoin_19': 'gisjoin_1940',
 'hispanic': 'hispanic',
 'hispanic_1': 'hispanic_perc_norm',
 'hispanic_p': 'hispanic_perc',
 'holc_grade': 'holc_grade',
 'homes': 'homes',
 'ln_homeval': 'ln_homeval_norm',
 'ln_income_': 'ln_income_norm',
 'ln_media_1': 'ln_median_homevalue_adj',
 'ln_median_': 'ln_median_income',
 'map_id': 'map_id',
 'median_h_1': 'median_homevalue_adj',
 'median_hom': 'median_homevalue',
 'median_i_1': 'median_income',
 'median_inc': 'median_income_adj',
 'other': 'other',
 'other_perc': 'other_perc',
 'owned_pe_1': 'owned_perc_norm',
 'owned_perc': 'owned_perc',
 'populati_1': 'population_density',
 'populati_2': 'population_density_norm',
 'populati_3': 'population_norm',
 'population': 'population',
 'unemploy_1': 'unemployed_perc',
 'unemploy_2': 'unemployed_perc_norm',
 'unemployed': 'unemployed',
 'white': 'white',
 'white_norm': 'white_norm',
 'white_pe_1': 'white_perc_norm',
 'white_perc': 'white_perc',
 'year': 'year'})


In [90]:
df_final['nonblack_perc'] = 1 - df_final['black_perc']/df_final['population']
df_final['perc_college_10plus'] = [1 if x > 0.10 else 0 for x in df_final['college_perc']]

# Need to create some extra features for the propensity scoring to increase similarity between 'before' tracts. 
df_final['perc_black_20plus'] = [1 if x > 0.20  else 0 for x in df_final['black_perc']]
df_final['perc_college_10plus'] = [1 if x > 0.10  else 0 for x in df_final['college_perc']]

# Create a point that represents roughly downtown is 
downtown = Point(-87.62, 41.855)
df_final['dist_downtown'] = df_final.distance(Point(-87.62,41.855))


Create different treatment periods

In [91]:
# there is no 1950?
print(df_final.shape)
print(df_final[df_final.year != 1950].shape)

(6289, 48)
(6289, 48)


In [92]:
df_final1 = df_final[df_final.year != 1950]
df_final1.loc[df_final1.year <= 1940,'period']='pre'
df_final1.loc[(df_final1.year > 1940) & (df_final1.year <= 1980), 'period']='post'

df_final1.loc[(df_final1.year>1980),'period']='reversal'

In [93]:
df_final1['fha_grade'].value_counts()

D    2541
C    1975
B    1117
A     656
Name: fha_grade, dtype: int64

In [94]:
# Create the D vs non-D FHA labels
df_final1['D_fha'] = df_final1.apply(lambda x: 'D' if x['fha_grade']=='D' else 'not_D_fha', axis=1)

In [95]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

def decade_scaler(x):
  scaler = MinMaxScaler()
  return scaler.fit_transform(x)

def standardize_data(df, features, groups = ['map_id', 'year']):
    for f in features: 
      # scaler = StandardScaler()
      # df['{}_norm'.format(f)] = df[]
      df['{}_norm'.format(f)] = df[groups + features].groupby(groups).transform(lambda x: (x-x.mean())/x.std())[f]
  
    return df

def clean_tracts(df):
    df = df.replace([np.inf, -np.inf], np.nan)

    df = df[(df['ln_median_homevalue_adj'] > 0) & (df['owned_perc'] > 0)]
    df = df.drop_duplicates(['gisjoin', 'year'])
    return df

In [10]:
df_final1['owned_perc']

0       0.035616
1       0.029748
2       0.077369
3       0.183099
4       0.187368
          ...   
6284    0.530171
6285    0.565171
6286    0.505742
6287    0.530852
6288    0.395582
Name: owned_perc, Length: 6289, dtype: float64

In [11]:
df_final1=clean_tracts(df_final1)

## Already did this earlier
# stand_features=['owned_perc','ln_median_homevalue_adj','white_perc','black_perc','unemployed_perc','asian_perc','hispanic_perc',
#                'population_density','population','ppl_per_home','dist_downtown']
# df_final = stand_data(df_final,stand_features)


In [12]:
df_final1['black'].describe()

count     6095.000000
mean      1008.967562
std       1835.188774
min          0.000000
25%          2.000000
50%         69.000000
75%       1336.500000
max      15307.000000
Name: black, dtype: float64

In [96]:
def total_E(df, feature='black_prop'):
    tot = df['population'].sum()
    p = (df[feature] * df['population']) / tot +.00000001
    p = p.sum()
    E = -p * np.log2(p) -\
        (1 - p) * np.log2(1-p)
        
    return E

In [97]:
y = 1940

W = ps.weights.Queen.from_dataframe(df_final1[df_final1.year == 1940],geom_col='geometry', idVariable='gisjoin')
W.transform = 'R'

d = df_final1
d['e'] = 1
d['e_w'] = 0
d['e_test'] = 0

#non-spatially dissimilarity measure
d.loc[d['year']==y,'e']

1       1
9       1
17      1
25      1
33      1
       ..
6251    1
6259    1
6267    1
6275    1
6282    1
Name: e, Length: 789, dtype: int64

In [68]:
df = pd.DataFrame(np.arange(10, 22).reshape(3, 4),
                  index=["a", "b", "c"],
                  columns=["A", "B", "C", "D"])
df['add'] = 0
df

,A,B,C,D,add
a,10,11,12,13,0
b,14,15,16,17,0
c,18,19,20,21,0


In [79]:
df.apply(lambda row:  row['A'] + 1)

KeyError: 'A'

In [112]:
def get_entropy(d): 
    
    '''
    Creates both the dissimiliarity measure and weighted entropy measure based on black 
    and non-black populations 
    '''
    
    d['e'] = 0
    d['e_w'] = 0
   
    for y in d['year'].unique():
        W = ps.weights.Queen.from_dataframe(d[d.year == 1940], geom_col='geometry', idVariable='gisjoin')
        
        #### Make sure the denominator is not zero
        d.loc[d['year']==y,'e'] = d.groupby('year').apply(lambda x: -x['black_perc'].values*np.log2(x['black_perc'].values+.000001)-\
                                          x['nonblack_perc'].values*np.log2(x['nonblack_perc'].values+.000001))[y]
        W.transform = 'R'
        
        d_yr = d[d.year == y]
        for gisjoin in d_yr['gisjoin'].values:

            w_black_prop = np.average(list(d_yr[d_yr['gisjoin'] == gisjoin]['black_perc'].values)+
                                      list(d_yr.loc[(d_yr['gisjoin'].isin(W.neighbors[gisjoin])),'black_perc']),
                                      weights=[1]+W.weights[gisjoin] )
            w_nonblack_prop = np.average(list(d_yr[d_yr.gisjoin==gisjoin]['nonblack_perc'].values)+
                                         list(d_yr.loc[(d_yr.gisjoin.isin(W.neighbors[gisjoin])),'nonblack_perc'].values),
                                         weights=[1]+ W.weights[gisjoin])
            try: 
                d.loc[(d['year']==y)&(d['gisjoin']==gisjoin),'e_w'] =-w_black_prop*np.log2(w_black_prop+.000001)-\
                                                w_nonblack_prop*np.log2(w_nonblack_prop+.000001)
                
            except (not W.neighbors[gisjoin]) or KeyError:
                print(gisjoin)
                d.loc[(d['year']==y)&(d['gisjoin']==gisjoin),'e_w'] = d.loc[(d['year']==y)&(d['gisjoin']==gisjoin),'e']
    return d

In [114]:
W.weights

{0: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 1: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 2: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 3: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 4: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 5: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 6: [1.0, 1.0, 1.0, 1.0, 1.0],
 7: [1.0, 1.0, 1.0, 1.0, 1.0],
 8: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 9: [1.0, 1.0, 1.0, 1.0],
 10: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 11: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 12: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 13: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 14: [1.0, 1.0, 1.0, 1.0],
 15: [1.0, 1.0, 1.0, 1.0, 1.0],
 16: [1.0, 1.0, 1.0, 1.0, 1.0],
 17: [1.0, 1.0],
 18: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 19: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 20: [1.0, 1.0, 1.0, 1.0, 1.0],
 21: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 22: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 23: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 24: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 25: [1.0, 1.0, 1.0, 1.0, 1.0],
 26: [1.0, 1.0, 1.0, 1.0, 1.

In [108]:
df_final2

,asian,asian_perc,college,college_perc,black,black_perc,gisjoin,gisjoin_1940,hispanic,hispanic_perc,...,fha_grade,geometry,nonblack_perc,perc_college_10plus,perc_black_20plus,dist_downtown,period,D_fha,e,e_w
0,0.000000,0.000000,0.000000,0.000000,10.000000,0.007704,G17003100609,G17003100609,0.000000,0.000000,...,A,"POLYGON ((-87.59668 41.80092, -87.59683 41.800...",0.999994,0,0,0.055883,pre,not_D_fha,0.054090,0.059802
1,0.000000,0.000000,288.000000,0.209913,11.000000,0.008017,G17003100609,G17003100609,0.000000,0.000000,...,A,"POLYGON ((-87.59668 41.80092, -87.59683 41.800...",0.999994,1,0,0.055883,pre,not_D_fha,0.055828,0.065039
2,0.000000,0.000000,296.000000,0.195122,1320.000000,0.870138,G17003100609,G17003100609,0.000000,0.000000,...,A,"POLYGON ((-87.59668 41.80092, -87.59683 41.800...",0.999426,1,1,0.055883,post,not_D_fha,0.175447,0.287579
3,4.000000,0.003344,282.000000,0.235786,976.000000,0.816054,G17003100609,G17003100609,12.000000,0.010033,...,A,"POLYGON ((-87.59668 41.80092, -87.59683 41.800...",0.999318,1,1,0.055883,post,not_D_fha,0.240301,0.351206
4,18.000000,0.019934,329.000000,0.364341,748.000000,0.828350,G17003100609,G17003100609,17.000000,0.018826,...,A,"POLYGON ((-87.59668 41.80092, -87.59683 41.800...",0.999083,1,1,0.055883,post,not_D_fha,0.226372,0.339153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6284,22.994468,0.007229,166.177861,0.058284,748.122434,0.292453,G17003100672,G17003100672,185.147045,0.058605,...,C,"POLYGON ((-87.59280 41.73316, -87.59301 41.733...",0.999902,0,1,0.119778,post,not_D_fha,0.518869,0.445012
6285,3.337476,0.000889,419.320257,0.121676,3207.218737,0.906346,G17003100672,G17003100672,79.461745,0.020452,...,C,"POLYGON ((-87.59280 41.73316, -87.59301 41.733...",0.999746,1,1,0.119778,post,not_D_fha,0.128943,0.088141
6286,2.000000,0.000694,717.669303,0.241117,2910.725257,0.968229,G17003100672,G17003100672,15.538262,0.005325,...,C,"POLYGON ((-87.59280 41.73316, -87.59301 41.733...",0.999679,1,1,0.119778,reversal,not_D_fha,0.045560,0.036299
6287,6.060534,0.001972,706.220508,0.232720,2902.857672,0.969141,G17003100672,G17003100672,27.188875,0.008474,...,C,"POLYGON ((-87.59280 41.73316, -87.59301 41.733...",0.999677,1,1,0.119778,reversal,not_D_fha,0.044290,0.039863
